# Basic 스태킹 모델

## 데이터 로드

In [2]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state= 216)

## 개별 Classifier와 최종 Stacking 데이터를 학습할 메타 Classifier 생성

In [3]:
# 개별 Classifier 모델 
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=216)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

# 최종 Stacking Classifier 모델
lr_final = LogisticRegression(C=10)

## 개별 Classifier 학습/예측/평가

In [4]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [5]:
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print(f"KNN 정확도: {accuracy_score(y_test, knn_pred):.4f}")
print(f"랜덤 포레스트 정확도: {accuracy_score(y_test, rf_pred):.4f}")
print(f"결정 트리 정확도: {accuracy_score(y_test, dt_pred):.4f}")
print(f"에이다부스트 정확도: {accuracy_score(y_test, ada_pred):.4f}")

KNN 정확도: 0.9561
랜덤 포레스트 정확도: 0.9737
결정 트리 정확도: 0.9298
에이다부스트 정확도: 0.9737


## 개별 모델 예측결과를 메타 모델이 학습하도록 스태킹 형태로 재 생성

In [6]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


## 메타 모델 학습/예측/평가

In [8]:
lr_final.fit(pred, y_test) # 데이터 세트 부족으로 테스트 데이터로 학습함 (오버피팅)
final = lr_final.predict(pred)

print(f"최종 메타 모델 예측 정확도: {accuracy_score(y_test, final):.4f}")

최종 메타 모델 예측 정확도: 0.9825


# CV 셋 기반의 Stacking

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터 생성 함수
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정 n_folds 값으로 KFold 생성
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=216)
    
    # 추후 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, 'model 시작')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        print(f'\n폴드 세트: {folder_counter} 시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # 폴드 세트 내부에서 만들어진 학습 데이터로 모델 학습 수행
        model.fit(X_tr, y_tr)
        
        # 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        
        # 원본 테스트 데이터를 폴드 세트의 학습된 모델에서 예측 후 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)
    
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    # train_fold_pred : 최종 메타 모델이 사용하는 학습데이터, test_pred_mean : 테스트 데이터
    return train_fold_pred, test_pred_mean

In [12]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7) # kfold=7
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier model 시작

폴드 세트: 0 시작

폴드 세트: 1 시작

폴드 세트: 2 시작

폴드 세트: 3 시작

폴드 세트: 4 시작

폴드 세트: 5 시작

폴드 세트: 6 시작
RandomForestClassifier model 시작

폴드 세트: 0 시작


E:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(



폴드 세트: 1 시작

폴드 세트: 2 시작

폴드 세트: 3 시작

폴드 세트: 4 시작

폴드 세트: 5 시작

폴드 세트: 6 시작


E:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
E:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


DecisionTreeClassifier model 시작

폴드 세트: 0 시작

폴드 세트: 1 시작

폴드 세트: 2 시작

폴드 세트: 3 시작

폴드 세트: 4 시작

폴드 세트: 5 시작

폴드 세트: 6 시작
AdaBoostClassifier model 시작

폴드 세트: 0 시작

폴드 세트: 1 시작

폴드 세트: 2 시작

폴드 세트: 3 시작

폴드 세트: 4 시작

폴드 세트: 5 시작

폴드 세트: 6 시작


In [14]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print("원본 학습 피쳐 데이터 Shape: ",X_train.shape, '원본 테스트 피쳐 Shape: ', X_test.shape)
print("스태킹 학습 피쳐 데이터 Shape: ", Stack_final_X_train.shape,
      "스태킹 테스트 피쳐 데이터 Shape: ", Stack_final_X_test.shape)

원본 학습 피쳐 데이터 Shape:  (455, 30) 원본 테스트 피쳐 Shape:  (114, 30)
스태킹 학습 피쳐 데이터 Shape:  (455, 4) 스태킹 테스트 피쳐 데이터 Shape:  (114, 4)


In [16]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print(f"최종 메타 모델의 예측 정확도 : {accuracy_score(y_test, stack_final):.4f}")

최종 메타 모델의 예측 정확도 : 0.9825
